## Alterações a serem feitas:
+ Criar tag de inicio e fim de arquivo
+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


os.chdir("mock")

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*/**/***/****.{}'.format(extension))]

In [3]:
all_filenames[0]

'161704902/[PRATICA_ETAPA_1]/Documentos/20180510_Rcl_22328_314302526.ner.csv'

# Teste

# Concatena todos os arquivos csv encontrados dentro de 'mock'
combined_csv = pd.concat([pd.read_csv(f,delimiter=';', na_values='NaN') for f in all_filenames])
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [40]:
df_teste = pd.read_csv(all_filenames[0], delimiter=';', na_values='NaN')
df_teste.head()

,Token,Tag
0,Ementa,O
1,,O
2,e,O
3,,O
4,Acórdão,O


In [101]:
list(df_teste['Token'])
len(df_teste.loc[df_teste['Token'] == '\n']), len(df_teste['Token'])

(553, 28813)

In [207]:
# Exibe 12 linhas antes e depois do primeiro padrão reconhecido para fim de contexto.

# i = 0
# df_teste['Token'].iloc[starts[i]-12:starts[i]+12]

In [176]:
# df_teste.join(:starts[i])
# pd.concat(df_teste['Token'][:starts[i]])
# df_teste['Token'].iloc(df_teste['Token'].index == starts[0])

In [ ]:
# for i in range(len(starts)):
#     df.iloc[i:starts[0]]

In [208]:
# i = 0
# df_teste.iloc[i:starts[i]]

# df_teste['Sentence #'] = 'Sentence'
# df_teste['Sentence #'][i:starts[i]] = 'Sentence %d'%(i+1) 
# df_teste[:15]

In [ ]:
# i = 1
# df_teste['Sentence #'][starts[i-1]:starts[i]] = 'Sentence %d'%(i+1) 
# df_teste[9:30]

In [205]:
# starts[:3], starts[-1], len(starts)+1

(Int64Index([12, 24, 98], dtype='int64'), 28799, 276)

In [209]:
df_teste['Sentence #'] = 'Sentence'
for i in range(0,len(starts)):
    df_teste['Sentence #'][:starts[0]] = 'Sentence %d'%(1) # Primeira sentença
    df_teste['Sentence #'][starts[i-1]:starts[i]] = 'Sentence %d'%(i+1) 
    df_teste['Sentence #'][starts[-1]:] = 'Sentence %d'%(len(starts)+1) # Última sentença
df_teste.head()

,Token,Tag,Sentence #
0,Ementa,O,Sentence 1
1,,O,Sentence 1
2,e,O,Sentence 1
3,,O,Sentence 1
4,Acórdão,O,Sentence 1


# Cria uma tag de inicio e fim de arquivo em cada arquivo antes de apendar todos os arquivos

In [4]:
frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN')
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ']
    frames.append(df)
    
combined_csv = pd.concat(frames)
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [5]:
combined_csv.head() , combined_csv.tail()
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 967910


In [16]:
combined_csv.reset_index(inplace=True, drop=True)

In [17]:
combined_csv[-10:]

,Token,Tag
967900,Chefe,O
967901,,O
967902,do,O
967903,,O
967904,Plenário,O
967905,\n,O
967906,id,O
967907,:,O
967908,,O
967909,20141203_ADI_4350_285683668,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [57]:
a_df = combined_csv #Simplifica o nome do arquivo 
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
# padroes = starts.unique()[1::2] # Pega o índice do segundo '\n' para marcar fim de sentença
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 9708


In [49]:
combined_csv.iloc[:starts[0]+2] # Primeira sentença
combined_csv.iloc[starts[-1]+2:] # Última sentença

,Token,Tag
967898,Assessora,O
967899,-,O
967900,Chefe,O
967901,,O
967902,do,O
967903,,O
967904,Plenário,O
967905,\n,O
967906,id,O
967907,:,O


In [47]:
i = 1
combined_csv.iloc[starts[i-1]+2:starts[i]+2]

,Token,Tag
14,RECLAMAÇÃO,O
15,,O
16,22.328,O
17,,O
18,RIO,O
19,,O
20,DE,O
21,,O
22,JANEIRO,O
23,,O


In [54]:
i = 2
combined_csv.iloc[starts[i-1]+2:starts[i]+2]

,Token,Tag
26,RELATOR,O
27,,O
28,:,O
29,,O
30,MIN,B_Pessoa
31,.,I_Pessoa
32,,I_Pessoa
33,ROBERTO,I_Pessoa
34,,I_Pessoa
35,BARROSORECLTE,O


In [152]:
import time

start = time.time()
combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 
combined_csv.head(), combined_csv.tail()
end = time.time()

In [165]:
elapsed_time = time.time() - start
'tempo',time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

('tempo', '00:18:28')

# Modelo

In [32]:
X = combined_csv.drop('Tag', axis=1) # Define o conjunto X
v = DictVectorizer(sparse=True) # Função que transforma listas de features em vetores
X = v.fit_transform(X.to_dict('records')) #Aplica a função de vetorização no conjunto 
                                          #X que foi colocado no formato 'records' (informa o que preenche cada coluna 
                                          # da linha i)
y = combined_csv.Tag.values # Define o conjunto y

classes = np.unique(y) # Define quais serão as classes baseado nos valores únicos da coluna y
classes = classes.tolist() # Tranforma as classes de array para lista

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0) # Divide o conjunto em treino
                                                                                            #e teste
X_train.shape, y_train.shape # Formato dos dados

((648499, 26420), (648499,))

In [33]:
classes

['B_Doutrina',
 'B_Doutrinador',
 'B_Pessoa',
 'B_Precedente',
 'B_Ref. Legislativa',
 'FIM_ARQ',
 'INICIO_ARQ',
 'I_Doutrina',
 'I_Doutrinador',
 'I_Pessoa',
 'I_Precedente',
 'I_Ref. Legislativa',
 'O']

In [34]:
def word2features(doc, i):
#     word = doc.iloc[i][0]
#     postag = doc.iloc[i][1]
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
#         word1 = doc.iloc[i+1][0]
#         postag1 = doc.iloc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features
# Agrupar arquivos

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
#     return [label for token, label, postag in sent]
    return [label for token, label in sent]
def sent2tokens(sent):
#     return [token for token, label, postag in sent]
    return [token for token, label in sent]

In [35]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [153]:
# Removendo a tag 'O'
new_classes = classes.copy()
new_classes.pop()
new_classes

['B_Doutrina',
 'B_Doutrinador',
 'B_Pessoa',
 'B_Precedente',
 'B_Ref. Legislativa',
 'FIM_ARQ',
 'INICIO_ARQ',
 'I_Doutrina',
 'I_Doutrinador',
 'I_Pessoa',
 'I_Precedente',
 'I_Ref. Legislativa']

In [157]:
# Separa as frases para criar contexto na aprendizagem

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
#         agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
#                                                            s['POS'].values.tolist(), 
#                                                            s['Tag'].values.tolist())]
        agg_func = lambda s: [(w, t) for w, t in zip(s['Token'].values.tolist(),
                                                     s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Token').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [158]:
getter = SentenceGetter(combined_csv)
sentences = getter.sentences

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [159]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    all_possible_transitions=False)

crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

1.0

In [132]:
[df['Token'][0], df['Tag'][0]]

['SEGURO', 'INICIO_ARQ']

# Processing dump

In [6]:
df_teste = pd.read_csv(all_filenames[0],delimiter=';', na_values='NaN')
df_teste.head()

,Token,Tag
0,Ementa,O
1,,O
2,e,O
3,,O
4,Acórdão,O


In [23]:
df_teste['Tag'].unique()

array(['O', 'B_Pessoa', 'I_Pessoa', 'B_Precedente', 'I_Precedente',
       'B_Ref. Legislativa', 'I_Ref. Legislativa', 'B_Doutrina',
       'I_Doutrina'], dtype=object)

In [25]:
'INICIO_ARQ', 'FIM_ARQ'

('INICIO_ARQ', 'FIM_ARQ')

In [26]:
df_teste['Tag'].iloc[0] , df_teste['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ']
df_teste.head(), df_teste.tail()

(     Token         Tag
 0   Ementa  INICIO_ARQ
 1                    O
 2        e           O
 3                    O
 4  Acórdão           O,                               Token      Tag
 28808                            \n        O
 28809                            id        O
 28810                             :        O
 28811                                      O
 28812  20180510_Rcl_22328_314302526  FIM_ARQ)

In [27]:
combined_csv.head()

,Token,Tag
0,Ementa,O
1,,O
2,e,O
3,,O
4,Acórdão,O
